<a href="https://colab.research.google.com/github/2025-02-FML-team/WV-Team/blob/main/notebooks/01_data_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import os 
import pandas as pd
import requests
from urllib.parse import urlparse
from pathlib import Path

data_dir = Path(os.path.join(os.getcwd(), "../data/")).resolve()
data_dir

PosixPath('/Volumes/Backup/Workspace/ML/WV-Team/data')

In [31]:
csv_url = data_dir / 'whisky_full.csv'

In [32]:
csv_url

PosixPath('/Volumes/Backup/Workspace/ML/WV-Team/data/whisky_full.csv')

In [33]:
# read the CSV file located in data/whiskies.csv
df = pd.read_csv(csv_url)

df = df.drop(['name_eng-2', 'category_specific'], axis=1)

# show the first few rows
df.head()

,id,name,name_eng,category,volume,alcohol,price,brand,nation,bottled,region,region_detail,distillery,importer,external_image_url,image_url,full_image_url
0,63800001,1423 S.B.S 가이아나 2020,1423 S.B.S Guyana 2020,기타,700,66.4,124900,1423 S.B.S,가이아나,NaN,NaN,NaN,NaN,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,NaN,NaN
1,11201698,카발란 클래식 싱글몰트,Kavalan Classic Single Malt,싱글몰트,700,40.0,144000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,/m/_next/image?url=https%3A%2F%2Fd1e2y5wc27crn...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
2,11201699,카발란 콘서트마스터 포트 캐스크 피니시,Kavalan Concertmaster Port Cask Finish,싱글몰트,700,40.0,173000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
3,11201700,카발란 콘서트마스터 포트 캐스크 피니시 1L,Kavalan Concertmaster Port Cask Finish 1L,싱글몰트,1000,40.0,200000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
4,11201701,카발란 콘서트마스터 셰리 캐스크 피니시,Kavalan Concertmaster Sherry Cask Finish,싱글몰트,700,40.0,204000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...


In [39]:
save_dir = os.path.join(os.getcwd(), "../data/photos")
os.makedirs(save_dir, exist_ok=True)

def get_extension_from_url(url):
    """Extract extension from URL if possible, else use .jpg"""
    path = urlparse(url).path
    ext = os.path.splitext(path)[1]
    if ext.lower() in [".jpg", ".jpeg", ".png", ".webp"]:
        return ext
    return ".jpg"

def download_image_if_not_exists(url, dest_path):
    """Download a single image safely"""
    try:
        if os.path.exists(dest_path):
            #print(f"✅ Already Exists {dest_path}")
            pass
        else:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            with open(dest_path, "wb") as f:
                f.write(resp.content)
            #print(f"✅ Saved {dest_path}")
    except Exception as e:
        print(f"❌ Failed {url[:30]}...")

all_paths = []

# Iterate through each row
for _, row in df.iterrows():
    whisky_id = row["id"]

    urls = [row.get("external_image_url"), row.get("full_image_url")]
    for idx, url in enumerate(urls, start=1):
        if pd.isna(url) or not isinstance(url, str):
            continue

        ext = get_extension_from_url(url)
        dest_path = os.path.join(save_dir, f"{whisky_id}_{idx}{ext}")

        download_image_if_not_exists(url, dest_path)
    all_paths.append(data_dir / f"{whisky_id}_1.webp")

df['local_full_path'] = all_paths

❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m

In [40]:
# Drop rows with NaN in full_image_url
filtered = df.dropna(subset=["full_image_url"])

# Drop rows where external_image_url starts with "/m/"
filtered = filtered[~filtered["external_image_url"].astype(str).str.startswith("/m/")]

print(f"Before: {len(df)} rows")
print(f"After : {len(filtered)} rows")

Before: 2958 rows
After : 2145 rows


In [41]:
for col in ["category", "category_specific"]:
    if col in df.columns:
        print(f"\n {col} 고유값 ({df[col].nunique()}개):")
        print(df[col].value_counts())
    else:
        print(f"{col} 컬럼이 없음")


 category 고유값 (6개):
category
싱글몰트    1261
기타       742
블렌디드     496
버번       232
브랜디      151
라이        76
Name: count, dtype: int64
category_specific 컬럼이 없음


In [42]:
#better to use english!!
category_map = {
    "싱글몰트": "Single Malt",
    "블렌디드": "Blended",
    "버번": "Bourbon",
    "브랜디": "Brandy",
    "라이": "Rye",
    "기타": "Other"
}

filtered["category"] = filtered["category"].map(category_map)

In [43]:
#Save the data just filtered
filtered_csv_url = os.path.join(os.getcwd(), "../data/whiskies_filtered.csv")

filtered.to_csv(filtered_csv_url, index=False)

In [44]:
filtered.head()

,id,name,name_eng,category,volume,alcohol,price,brand,nation,bottled,region,region_detail,distillery,importer,external_image_url,image_url,full_image_url,local_full_path
2,11201699,카발란 콘서트마스터 포트 캐스크 피니시,Kavalan Concertmaster Port Cask Finish,Single Malt,700,40.0,173000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,/Volumes/Backup/Workspace/ML/WV-Team/data/1120...
3,11201700,카발란 콘서트마스터 포트 캐스크 피니시 1L,Kavalan Concertmaster Port Cask Finish 1L,Single Malt,1000,40.0,200000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,/Volumes/Backup/Workspace/ML/WV-Team/data/1120...
4,11201701,카발란 콘서트마스터 셰리 캐스크 피니시,Kavalan Concertmaster Sherry Cask Finish,Single Malt,700,40.0,204000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,/Volumes/Backup/Workspace/ML/WV-Team/data/1120...
6,11201703,카발란 킹 카 컨덕터,Kavalan King Car Conductor,Single Malt,700,46.0,190000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,/Volumes/Backup/Workspace/ML/WV-Team/data/1120...
9,11201706,카발란 포디움,Kavalan Podium,Single Malt,700,46.0,229000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,/Volumes/Backup/Workspace/ML/WV-Team/data/1120...
